In [ ]:
import pickle
from google.colab import drive
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS   = 950


print("데이터 로딩 중...")
drive.mount('/content/drive')
file_path = "/content/drive/My Drive/Colab Notebooks/mon_standard.pkl"
with open(file_path, 'rb') as fi:
    data = pickle.load(fi)

데이터 로딩 중...
Mounted at /content/drive


In [ ]:
# 데이터 전처리
X_in_num = []
y = []
for i in range(TOTAL_URLS):
    if USE_SUBLABEL:
        label = i
    else:
        label = i // URL_PER_SITE
    for sample in data[i]:
        in_num = 0
        for index, c in enumerate(sample):
            dr = 1 if c > 0 else -1
            if dr == -1:
                in_num += 1;
        X_in_num.append(in_num)
        y.append(label)


# 2D 배열로 변환
X_in_num_array = np.array(X_in_num).reshape(-1, 1)

# 데이터를 학습 및 테스트 세트로 나눔
X_train_in_num, X_test_in_num, y_train_in_num, y_test_in_num = train_test_split(X_in_num_array, y, test_size=0.2, random_state=42)

# StandardScaler를 사용하여 X_in_num 데이터 정규화
scaler = StandardScaler()
X_train_in_num_scaled = scaler.fit_transform(X_train_in_num)
X_test_in_num_scaled = scaler.transform(X_test_in_num)

In [ ]:
#svm
svm_model_in_num = SVC(kernel='linear', C=1.0, random_state=42)
svm_model_in_num.fit(X_train_in_num_scaled, y_train_in_num)

y_pred_svm_in_num = svm_model_in_num.predict(X_test_in_num_scaled)
accuracy_svm_in_num = accuracy_score(y_test_in_num, y_pred_svm_in_num)

print(f"svm 모델 정확도 (X_in_num): {accuracy_svm_in_num * 100:.2f}%")

svm 모델 정확도 (X_in_num): 4.74%


In [ ]:
from sklearn.model_selection import GridSearchCV
# 튜닝할 하이퍼파라미터 설정
param_grid = {
    'C': [10, 100],  # C 값 튜닝
    'gamma': [0.1, 1.0],  # gamma 값 튜닝
    'kernel': ['linear', 'rbf'],  # 커널 튜닝
}

# 그리드 서치를 사용하여 최적의 파라미터 찾기
grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=5)
grid_search.fit(X_train_in_num_scaled, y_train_in_num)

# 최적의 파라미터 출력
print("최적의 하이퍼파라미터:", grid_search.best_params_)

# 최적의 모델 얻기
best_svm_model_in_num = grid_search.best_estimator_

# 최적의 모델로 예측 및 정확도 계산
y_pred_best_svm_in_num = best_svm_model_in_num.predict(X_test_in_num_scaled)
accuracy_best_svm_in_num = accuracy_score(y_test_in_num, y_pred_best_svm_in_num)
print(f"튜닝 후 SVM 모델 정확도 (X_in_num): {accuracy_best_svm_in_num * 100:.2f}%")

최적의 하이퍼파라미터: {'C': 100, 'gamma': 1.0, 'kernel': 'rbf'}
튜닝 후 SVM 모델 정확도 (X_in_num): 11.74%


In [ ]:
# rf
rf_model_in_num = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_in_num.fit(X_train_in_num_scaled, y_train_in_num)

y_pred_rf_in_num = rf_model_in_num.predict(X_test_in_num_scaled)
accuracy_rf_in_num = accuracy_score(y_test_in_num, y_pred_rf_in_num)
print(f"랜덤 포레스트 모델 정확도 (in_num): {accuracy_rf_in_num * 100:.2f}%")

랜덤 포레스트 모델 정확도 (in_num): 14.92%


In [ ]:
# 튜닝할 하이퍼파라미터 설정
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
}

# 그리드 서치를 사용하여 최적의 파라미터 찾기
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5)
grid_search_rf.fit(X_train_in_num_scaled, y_train_in_num)

# 최적의 파라미터 출력
print("최적의 하이퍼파라미터:", grid_search_rf.best_params_)

# 최적의 모델 얻기
best_rf_model_in_num = grid_search_rf.best_estimator_

# 최적의 모델로 예측 및 정확도 계산
y_pred_best_rf_in_num = best_rf_model_in_num.predict(X_test_in_num_scaled)
accuracy_best_rf_in_num = accuracy_score(y_test_in_num, y_pred_best_rf_in_num)
print(f"튜닝 후 랜덤 포레스트 모델 정확도 (in_num): {accuracy_best_rf_in_num * 100:.2f}%")

최적의 하이퍼파라미터: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
튜닝 후 랜덤 포레스트 모델 정확도 (in_num): 21.03%


In [ ]:
# knn
knn_model_in_num = KNeighborsClassifier(n_neighbors=5)
knn_model_in_num.fit(X_train_in_num_scaled, y_train_in_num)

y_pred_knn_in_num = knn_model_in_num.predict(X_test_in_num_scaled)

accuracy_knn_in_num = accuracy_score(y_test_in_num, y_pred_knn_in_num)
print(f"KNN 모델 정확도 (X_in_num): {accuracy_knn_in_num * 100:.2f}%")

KNN 모델 정확도 (X_in_num): 16.55%


In [ ]:
# 튜닝할 하이퍼파라미터 설정
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],  # 이웃의 개수 튜닝
}

# 그리드 서치를 사용하여 최적의 파라미터 찾기
grid_search_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5)
grid_search_knn.fit(X_train_in_num_scaled, y_train_in_num)

# 최적의 파라미터 출력
print("최적의 하이퍼파라미터:", grid_search_knn.best_params_)

# 최적의 모델 얻기
best_knn_model_in_num = grid_search_knn.best_estimator_

# 최적의 모델로 예측 및 정확도 계산
y_pred_best_knn_in_num = best_knn_model_in_num.predict(X_test_in_num_scaled)
accuracy_best_knn_in_num = accuracy_score(y_test_in_num, y_pred_best_knn_in_num)
print(f"튜닝 후 KNN 모델 정확도 (X_in_num): {accuracy_best_knn_in_num * 100:.2f}%")

최적의 하이퍼파라미터: {'n_neighbors': 9}
튜닝 후 KNN 모델 정확도 (X_in_num): 18.18%
